In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV, KFold
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

data = pd.read_csv('TrainingData.csv')  


spain_coords = {'lat_min': 36.0, 'lat_max': 43.0, 'lon_min': -9.0, 'lon_max': 3.0}
kenya_coords = {'lat_min': -4.5, 'lat_max': 5.0, 'lon_min': 34.0, 'lon_max': 42.0}
vnm_coords = {'lat_min': 8.0, 'lat_max': 24.0, 'lon_min': 102.0, 'lon_max': 110.0}

# Function to filter data for a specific country
def filter_data(data, coords):
    return data[(data['lat'] >= coords['lat_min']) & (data['lat'] <= coords['lat_max']) & 
                (data['lon'] >= coords['lon_min']) & (data['lon'] <= coords['lon_max'])]


data_spain = filter_data(data, spain_coords)
data_kenya = filter_data(data, kenya_coords)
data_vnm = filter_data(data, vnm_coords)

# Combine the filtered data
filtered_data = pd.concat([data_spain, data_kenya, data_vnm])

# Define features and target
features = ['lon', 'lat', 'blue_p50', 'green_p50', 'nir_p50', 'nira_p50', 're1_p50', 're2_p50', 're3_p50', 'red_p50', 'swir1_p50', 'swir2_p50', 'VV_p50', 'VH_p50']
target = 'TARGET'

X = filtered_data[features]
y = filtered_data[target]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


/home/chris/miniconda3/envs/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

param_grid_lgbm = {
    'n_estimators': [300],  # Already found optimal
    'num_leaves': [83],     # Already found optimal
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [10, 15, 20],
    'min_child_samples': [5, 10, 20],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [0.0, 0.1, 0.5],
    'reg_lambda': [0.0, 0.1, 0.5]
}

grid_search_lgbm = GridSearchCV(
    LGBMClassifier(random_state=42),
    param_grid_lgbm,
    cv=KFold(n_splits=10, shuffle=True, random_state=42),
    scoring='accuracy',
    n_jobs=-1
)


grid_search_lgbm.fit(X_train, y_train)

best_params_lgbm = grid_search_lgbm.best_params_
best_score_lgbm = grid_search_lgbm.best_score_

print(f"Best parameters for LightGBM: {best_params_lgbm}")
print(f"Best cross-validation accuracy for LightGBM: {best_score_lgbm}")

# Evaluate the best model on the validation set
best_model_lgbm = grid_search_lgbm.best_estimator_
val_accuracy_lgbm = best_model_lgbm.score(X_val_scaled, y_val)

print(f"Validation accuracy of the best model: {val_accuracy_lgbm}")


# classifiers = {
#     'lightgbm': LGBMClassifier(random_state=42),
# }

# param_grid = {
#     'lightgbm': {
#         'n_estimators': [300, 500],
#         'num_leaves': [83, 100]
#     }
# }
# grid_search_results = {}


# for clf_name, clf in classifiers.items():
#     # Set the classifier in the pipeline   
#     # Create the grid search object
#     grid_search = GridSearchCV(clf, param_grid[clf_name], cv=KFold(n_splits=10, shuffle=True, random_state=42), scoring='accuracy', n_jobs=-1)
    
#     # Fit the grid search
#     grid_search.fit(X_train, y_train)
    
#     # Store the best estimator and the best score
#     grid_search_results[clf_name] = {
#         'best_estimator': grid_search.best_estimator_,
#         'best_score': grid_search.best_score_
#     }
    
#     print(f"Best parameters for {clf_name}: {grid_search.best_params_}")
#     print(f"Best cross-validation accuracy for {clf_name}: {grid_search.best_score_}")

# # Evaluate the best model on the validation set
# best_model = max(grid_search_results.items(), key=lambda x: x[1]['best_score'])[1]['best_estimator']
# val_score = best_model.score(X_val, y_val)
# print(f"Validation accuracy of the best model ({best_model}): {val_score}")

# # # Save the best model and the label encoder
# # joblib.dump(best_model, 'best_model_pipeline.pkl')

/home/chris/miniconda3/envs/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/chris/miniconda3/envs/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/chris/miniconda3/envs/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/chris/miniconda3/envs/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.

[LightGBM] [Info] Number of positive: 1066, number of negative: 968
[LightGBM] [Info] Number of positive: 1058, number of negative: 976
[LightGBM] [Info] Number of positive: 1058, number of negative: 976
[LightGBM] [Info] Number of positive: 1084, number of negative: 950
[LightGBM] [Info] Number of positive: 1063, number of negative: 971
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032648 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Number of positive: 1068, number of negative: 966
[LightGBM] [Info] Number of positive: 1058, number of negative: 976
[LightGBM] [Info] Total Bins 3570
[LightGBM] [Info] Number of positive: 1065, number of negative: 969
[LightGBM] [Info] Number of data points in the train set: 2034, number of used features: 14
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055308 seconds.
You can set `force_col_wise=true` to remove the overhead.
[Light

In [ ]:
import pandas as pd

def add_id_column(df, file_name):
  """Adds an ID column based on the file name and index."""
  file_prefix = file_name.split('_')[0]
  df['ID'] = df.index.map(lambda x: f"{file_prefix}_{x+1}")
  return df

t1 = pd.read_csv('Kenya_testing.csv')
t2 = pd.read_csv('Spain_validation.csv')
t3 = pd.read_csv('VNM_testing.csv')


test1 = add_id_column(t1, 'Kenya')  
test2 = add_id_column(t2, 'Spain')
test3 = add_id_column(t3, 'VNM')

t1 = pd.DataFrame(test1)
t2 = pd.DataFrame(test2)
t3 = pd.DataFrame(test3)



In [ ]:
sample_submission = pd.read_csv('SampleSubmission.csv')
print(sample_submission.head())
test = pd.concat([t1, t2, t3], ignore_index=True, join='outer')
Test = pd.DataFrame(test)

merged_data = pd.merge(sample_submission, Test, on='ID', how='left')


In [ ]:
testdata = merged_data.drop(columns=['ID','TARGET'])

test_pred = forest_clf.predict(testdata)

# Create submission DataFrame
submission_pred = pd.DataFrame({'ID': sample_submission['ID'], 'TARGET': test_pred})

# Save to CSV
submission_pred.to_csv('pred_submission.csv', index=False)

submission_pred.head()
